In [16]:
import sys
import copy
import random
import hashlib
import cupy as cp 
import numpy as np
from math import pi
import numpy.linalg as LA
from scipy.integrate import RK45

from os.path import abspath, join
sys.path.append(abspath(join('..')))
sys.path.append(abspath(join('../..')))

from datetime import datetime
from os.path import expanduser

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from LevelSetPy.Grids import *
from LevelSetPy.Utilities import *
from LevelSetPy.Visualization import *
from LevelSetPy.DynamicalSystems import *
from LevelSetPy.BoundaryCondition import *
from LevelSetPy.InitialConditions import *
from LevelSetPy.SpatialDerivative import *
from LevelSetPy.ExplicitIntegration import *
from BRATVisualization.rcbrt_visu import RCBRTVisualizer

%matplotlib inline

In [8]:
args = Bundle(dict(visualize=True, flock_num=4, resume="murmurations_flock_01_02-06-22_17-43.hdf5", flock_payoff=False, pause_time=1e-5))

In [9]:
grid_min = expand(np.array((-.75, -1.25, -np.pi)), ax = 1)
grid_max = expand(np.array((3.25, 1.25, np.pi)), ax = 1)
pdDims = 2                      # 3rd dimension is periodic
resolution = 100
N = np.array(([[
                resolution,
                np.ceil(resolution*(grid_max[1, 0] - grid_min[1, 0])/ \
                            (grid_max[0, 0] - grid_min[0, 0])),
                resolution-1
                ]])).T.astype(int)
grid_max[2, 0] *= (1-2/N[2,0])
g = createGrid(grid_min, grid_max, N, pdDims)

### Two-Player Differential Game Between Two Rockets

At issue are two rocket missiles on a state space: (i) a pursuer rocket $P$ whose goal is to "capture" an "evader" rocket $E$ within a given time interval $[-T, 0]$ for $T>0$. This is akin to a war game where e.g. an iron dome must intercept a missile within a time period. Outside the _measure_  $T$, the evading rocket may land in a large civilian population and destroy a lot of properties. The pursuer is seeking to minimize the time of capture, while the evader is seeking to maximize the time of capture.

In what follows, we shall set up a two-player differential game system of equations between these two agents in absolute coordinates. We leverage Dreyfus' rocket launch example used by S.K. Mitter in his [Automatica paper from 1966](https://scholar.google.com/scholar_url?url=https://stuff.mit.edu/people/mitter/publications/1_successive_approx_AUTO.pdf&hl=en&sa=T&oi=gsb-gga&ct=res&cd=0&d=9474146906219145202&ei=lqGDYqvgNM6Uy9YPjP-C6A4&scisig=AAGBfm0MFGEhs6vvhQeFLR1e3EQvtwya3g).

Here are the key variables for the pursuer and evader respectively:


| $x_1/x_5$ (ft)     | $x_2/x_6$  (ft)    | $x_3/x_7$  (ft/sec)    |     $x_4/x_8$  (ft/sec) |
| :--:     | :---:               | :---:               |  :---:               | 
| Horizontal range    |      Altitude   |    Horizontal velocity    | Vertical velocity  |

The equations of motion are adopted from Dreyfus' construction as follows:

$$
\begin{align}
    &\dot{x}_1 = x_3, \,\, &x_1(-T) = 0; \qquad \qquad &\dot{x}_5 = x_7, \,\, &x_5(-T) = 0 \\
    &\dot{x}_2 = x_4, \,\, &x_2(-T) = 0; \qquad \qquad &\dot{x}_6 = x_8, \,\, &x_6(-T) = 0\\
    &\dot{x}_3 = a\cos(u), \,\, &x_3(-T) = 0; \qquad \qquad &\dot{x}_7 = a\cos(v), \,\, &x_7(-T) = 0 \\
    &\dot{x}_4 = a\sin(u) - g; \quad &x_4(-T) = 0; \qquad \qquad &\dot{x}_8 = a\sin(v) - g, \,\, &x_8(-T) = 0
\end{align}
$$

where $u(t), t \in [-T,0]$ is the controller under the coercion of the evader and $v(t), t \in [-T,0]$ is the controller under the coercion of the pursuer i.e. the pursuer is minimizing while the evader is maximizing. The full state dynamics is given by

$$
\begin{align}
 \dot{x} = \left(\begin{array}{c}
 x_3 \\
 x_4 \\
 a\cos(u) \\
 a\sin(u) - g \\
 x_7 \\
 x_8 \\
 a\cos(v) \\
 a\sin(v) - g
 \end{array}\right).
\end{align}
$$

**Assumptions**: To simply the differential game, assume $u(t)$ and $v(t)$ are drawn from the Lebesgue measurablre sets as given in the LCSS paper with $u(t) = [\underline{c},\bar{c}]$ and $v(t) = [\underline{c},\bar{c}]$ for a $\underline{c} = -1$ and a $\bar{c}=1$.


We want to construct a target set such that the [Reduced Vertical Separation Minimum](https://www.faa.gov/air_traffic/separation_standards/rvsm/) between the two rockets at any time $t \in [-T, 0]$ is at least $h=1000 ft$ and the horizontal separation between the two rockets at any time $t \in [-T, 0]$ is at least $w=1000 ft$. The RVSM we use is consistent with FAA recommendations for airborne systems. Capture occurs when the $l_2$ distance between the horizontal **or** vertical capture falls below the specified range given. We write the target set thus:  
 
 $$
 \mathcal{T}  = \{x \in \mathcal{X} \mid x_2^2 + x_6^2 \le h \,\, || \,\, x_1^2 + x_5^2 \le w \}
$$ where we have introduced the Boolean **OR** operator `||` to account for the capture scheme as enumerated above. Note that if we are using an implicit construction of the target set on the state  space, we would simply use the `min` operator.

We require $u(t)$ to maximize the time of capture and $v(t)$ to minimize the time of capture. By this logic, this is tantamount to $u(t)$ maximizing the velocity $x_3(t)$ and $v(t)$ minimizing $x_7(t)$.

Therefore, we can rewrite the target set and the _robustly controlled backward reachable tube_ for a value function $V: [-T,0]\times \mathcal{X} \rightarrow \mathbb{R}$ as 

$$
\begin{align}
 \mathcal{T}  &= \{x \in \mathcal{X} \mid V(x, 0) \le 0 \}, \\
 \ell([-t,0], \mathcal{T}) &=  \{x \in \mathcal{X} \mid V(x, t) \le 0 \}  \text{  for any  } t \in [0,T].
 \end{align}
$$
 


We write the zero-level set $V(x,0)$ from $\mathcal{T}$'s definition:

$$
\begin{align}
     V(x,0) = x_2^2 + x_6^2 - h + x_1^2 + x_5^2 - w
\end{align}
$$

so that at times $t >0$, the variational HJI PDE is 

$$
\begin{align}
     \dfrac{\partial V(x,t)}{\partial t} + \min \left\{0, \min_{v} \max_{u}H\left(x,u,v,\dfrac{\partial V(x,t)}{\partial x} \right)\right\} = 0.
\end{align}
$$

Whereupon the Hamiltonian is given by 

$$
\begin{align}
H(x,u,v,p) &=  \left(\left(\begin{array}{cc cc cc cc}p_1 & p_2 & p_3 & p_4 & p_5 & p_6 & p_7 & p_8  \end{array}\right) \left(\begin{array}{c}
 x_3 \\
 x_4 \\
 a\cos(u) \\
 a\sin(u) - g \\
 x_7 \\
 x_8 \\
 a\cos(v) \\
 a\sin(v) - g
 \end{array}\right)
 \right)
 \\
 &=  
 p_1 \, x_3 + p_2 \, x_4 + a \, p_3\cos(u) + p_4 \,\left( a\sin(u) - g \right) + p_5 \, x_7 + p_6 \, x_8 + a \, p_7\cos(v) + p_8 \, \left(a\sin(v) - g\right)
 \end{align}
$$


## Hamiltonian

$$H(x,u,v,p) = p_1 \, x_3 + p_2 \, x_4 + a \, p_3\cos(u) + p_4 \,\left( a\sin(u) - g \right) + p_5 \, x_7 + p_6 \, x_8 + a \, p_7\cos(v) + p_8 \, \left(a\sin(v) - g\right)$$
Partial derivative of $H$ with respect to $u$
$$
\begin{align}
&H_u = -p_3a\sin(u) + p_4a\cos(u)\\
&H_{uu} = -p_3a\cos(u) - p_4a\sin(u) \\
&H_{uv} = 0 \\
&H_{ux} = 0_{8 \times 1}
\end{align}
$$
Partial derivative of $H$ with respect to $v$
$$
\begin{align}
&H_v = -p_7a\sin(v) + p_8a\cos(v) \\
&H_{vv} = -p_7a\cos(v) - p_8a\sin(v) \\
&H_{vu} = 0 \\
&H_{vx} = 0_{8 \times 1}
\end{align}
$$

Partial derivative of $H$ with respect to $x$
$$
\begin{align}
&H_x = \begin{bmatrix}0 &0 &p_1 &p_2 &0 &0 &p_5 &p_6 \end{bmatrix}^T \\
&H_{xx} = 0_{8 \times 8} \\
&H_{xu} = 0_{8 \times 1}\\
&H_{xv} = 0_{8 \times 1}
\end{align}
$$

In [22]:
def system(x, u, v):
    '''
    dx/dt = f(x, u, v) 
    input:
        x - state
        u - control input (evader)
        v - disturbance input (pursuer)
    output:
        dx - derivative of state
        f_x f_u f_v- derivative of f with respect to x/u/v
    '''
    g = 32.17 # gravitational acceleration, 32.17ft/sec^2
    a = 100.0 # acceleration of the rocket
    
    dx = np.array([x[2], x[3], a*np.cos(u), a*np.sin(u)-g, x[6], x[7], a*np.cos(v), a*np.sin(v)-g])
    
    f_x = np.zeros([8,8])
    f_x[0,2] = 1 
    f_x[1,3] = 1
    f_x[4,6] = 1
    f_x[5,7] = 1
    
    f_u = np.zeros([8,1])
    f_u[2] = -a*np.sin(u)
    f_u[3] = a*np.cos(u)
    
    f_v = np.zeros([8,1])
    f_v[6] = -a*np.cos(v)
    f_v[7] = a*np.cos(v)
    
    return dx, f_x, f_u, f_v

def system_RK45(t, delta_x, x_r, u_r, v_r, k_u, k_v):   
    '''
    Created for forward pass
    input:
        t - time
        delta_x - variational of state
        x_r - normal state
        u_r - normal control input
        v_r - normal disturbance
        k_u - variational feedback controller for evader
        k_v - variational feedback controller for pursuer
    output:
    
    '''
    
    delta_u = k_u@delta_x
    delta_v = k_v@delta_x
    
    d_x_r, _, _, _ = system(x_r, u_r, v_r)
    d_delta_x, _, _, _  = system(x_r+delta_x, u_r+delta_u, v_r+delta_v)
    
    d_delta_x = d_delta_x - d_x_r
    
    return d_delta_x

def hamiltonian(x, u, v, p, a):
    '''
    input:         
        x - state
        u - control input
        v - disturbance
        p - costate
    output:
        H, H_u, H_ux, H_uu, H_v, H_vx, H_vv, H_uv, H_xx, H_xu, H_xv - derivatives of hamiltonian
    '''
    
    a = 100.0 # acceleration of the rocket
    
    H = p@np.array([x[2], x[3], a*np.cos(u), a*np.sin(u)-g, x[6], x[7], a*np.cos(v), a*np.sin(v)-g])
    
    H_u = -p[2]*a*np.sin(u) + p[3]*a*np.cos(u)         
    H_uu = -p[2]*a*np.cos(u) - p[3]*a*np.sin(u)    
    H_uv = np.array([0]) 
    H_ux = np.zeros(8)
    
    H_v = -p[6]*a*np.sin(v) + p[7]*a*np.cos(v)      
    H_vv = -p[6]*a*np.cos(v) - p[7]*d*np.sin(v) 
    H_vx = np.zeros(8)
    

    H_x = np.zeros(8)
    H_x[2] = p[0]
    H_x[3] = p[1]
    H_x[6] = p[4]
    H_x[7] = p[5]   
    H_xx = np.zeros([8, 8])

    return H, H_u, H_uu, H_uv, H_ux, H_v, H_vv, H_vx, H_x, H_xx

In [25]:
def TargetSetFunc(x):
    '''
    input:
        x - state
    output:
        the terminal cost 
    '''
    
    h = 1000.0 #Vertical seperation
    w = 1000.0 #Horizontal seperation
       
    return np.logical_or(x[0]**2+x[4]**2-w, x[1]**2-x[5]**2-h)
    

In [26]:
def forward(x_r, u_opt, v_opt, Val_opt, Vx_opt, Vxx_opt, t_grid):
    '''
    input: 
        x_r - trajectory of the normal states
        u_opt - trajectory of the optimal control (evader)
        v_opt - trajectory of the optimal disturbance (pursuer)
        t_grid - time grid partitions the time interval [-T, 0]
        Val_opt - optimal value function
        Vx_opt - gradient of optimal value function
        Vxx_opt - Hessian matrix of optimal value function
    output:
        Val_opt - updated optimal value function
        x_r - updated trajectory of the state
        u_r - updated trajectory of the nominal control
        v_r - updated trajectory of the nominal disturbance
    '''
    
    delta_x = np.zeros(8)
    
    for i in range(len(t_grid)-1):
                    
        t = t_grid[i]
        dx, f_x, f_u, f_v = system(x_r[i], u_opt[i], v_opt[i])
        H, H_u, H_uu, H_uv, H_ux, H_v, H_vv, H_vx, H_x, H_xx = hamiltonian(x_r[i], u_opt[i], v_opt[i], Vx_opt[i])
        
        k = np.linalg.inv(np.array([[H_uu, H_uv],[H_vu, H_vv]]))@(H_ux+f_u@Vxx_opt[i])
        k_u = k[0]
        k_v = k[1]
        
        t0 = t
        tf = t_grid[i+1]
        delta_x = RK45(system_RK45, t0, delta_x, tf, args=(x_r[i], u_r[i], v_r[i], k_u, k_v))
        
        Val_opt[i+1] = Val_opt[i+1] + delta_x@Vx_opt[i+1] + 0.5*delta_x@Vxx_opt[i+1]@delta_x
        
        u_r[i+1] = u_r[i+1] + k_u@delta_x
        v_r[i+1] = v_r[i+1] + k_v@delta_x
        
    return u_r, v_r, Val_opt
        
    
        

In [27]:
def backward(x_r, u_r, v_r, t_grid, ):
    '''
    input: 
        x_r - trajectory of the normal states
        u_r - trajectory of the normal control (evader)
        v_r - trajectory of the normal disturbance (pursuer)
        t_grid - time grid partitions the time interval [-T, 0]
    output:
        Val_opt - updated optimal value function
        Vx_opt - gradient of optimal value function
        Vxx_opt - Hessian matrix of optimal value function
    '''
    # The cost of the trajectory is equal to the value of TargetSetFunc when state first enters the target set 
    Val_r = TargetSetFunc(x_r[-1])  # Calculate the terminal cost of the trajectory     
    for i in range(len(t_grid)-1,-1,-1):
        if TargetSetFunc(x_r[i])<=0:
            Val_r = TargetSetFunc(x_r[i])
            
    
        
        
        
    
    
    

### Dynamics of the System

Here, the goal is to launch a rocket in fixed time to a given altitude and with a given final vertical velocity component and maximum final horizontal velocity component.

| $x_1/x_5$ (ft)     | $x_2/x_6$  (ft)    | $x_3/x_7$  (ft/sec)    |     $x_4/x_8$  (ft/sec) |
| :--:     | :---:               | :---:               |  :---:               | 
| Horizontal range    |      Altitude   |    Horizontal velocity    | Vertical velocity  |

$$
\begin{align}
    &\dot{x}_1 = x_3; \qquad \qquad &x_1(-T) = 0 \\
    &\dot{x}_2 = x_4; \qquad \qquad &x_2(-T) = 0 \\
    &\dot{x}_3 = a\cos(u) + d\cos(v); \qquad \qquad &x_3(-T) = 0 \\
    &\dot{x}_4 = a\sin(u) + d\sin(v) - g; \qquad \qquad &x_4(-T) = 0
\end{align}
$$

Understand that $g$ is the gravitational acceleration and is given as $32.17 ft/s^2$.

| $x_5$ (ft)     | $x_6$  (ft)    | $x_3$  (ft/sec)    |     $x_4$  (ft/sec) |
| :--:     | :---:               | :---:               |  :---:               | 
| Horizontal range    |      Altitude   |    Horizontal velocity    | Vertical velocity  |

$$
\begin{align}
    &\dot{x}_1 = x_3; \qquad \qquad &x_1(-T) = 0 \\
    &\dot{x}_2 = x_4; \qquad \qquad &x_2(-T) = 0 \\
    &\dot{x}_3 = a\cos(u) + d\cos(v); \qquad \qquad &x_3(-T) = 0 \\
    &\dot{x}_4 = a\sin(u) + d\sin(v) - g; \qquad \qquad &x_4(-T) = 0
\end{align}
$$

Goal: Choose $u(t); t \in [-T, 0]$ to maximize the horizontal velocity $x_3(0)$ while choosing $v(t); t \in [-T,0]$  to minimize the vertical velocity $x_4(0)$.


Write it as compact form
$$
\begin{align}
\dot{x} = f(x) + l(u) + g(v)
\end{align}
$$
Objective:
$$
\max_{u} \,\, \min_{v} J(x_0, u, v) = x_3(t_f) - x_4(t_f)
$$

Define $g(x) = x_3 - x_4 + C$, where $C\in \mathbb{R}$ is a constant. Then, the target set is
$$
\mathcal{L}_0 = \{x \in \mathbb{R}^4| g(x) \leq 0 \}
$$

## Hamiltonian

$$H(x,u,v,p) = p_1x_3 + p_2x_4 + p_3(a\cos(u) + d\cos(v)) + p_4(a\sin(u)+d\sin(v)-g)$$
Partial derivative of $H$ with respect to $u$
$$
\begin{align}
&H_u = -p_3a\sin(u) + p_4a\cos(u) \\
&H_{uu} = -p_3a\cos(u) - p_4a\sin(u) \\
&H_{uv} = 0 \\
&H_{ux} = \begin{bmatrix}0 &0 &0 &0 \end{bmatrix}^T
\end{align}
$$
Partial derivative of $H$ with respect to $v$
$$
\begin{align}
&H_v = -p_3d\sin(v) + p_4d\cos(v) \\
&H_{vv} = -p_3d\cos(v) - p_4d\sin(v) \\
&H_{vu} = 0 \\
&H_{vx} = \begin{bmatrix}0 &0 &0 &0 \end{bmatrix}^T
\end{align}
$$

Partial derivative of $H$ with respect to $x$
$$
\begin{align}
&H_x = \begin{bmatrix}0 &0 &p_1 &p_2 \end{bmatrix}^T \\
&H_{xx} = 0_{4\times4} \\
&H_{xu} = \begin{bmatrix}0 &0 &0 &0 \end{bmatrix}^T\\
&H_{xv} = \begin{bmatrix}0 &0 &0 &0 \end{bmatrix}^T
\end{align}
$$

In [6]:
def hamiltonian(a, b, x, u, v, p):
    '''
    input: 
        a/b - acceleration of propulsive force/wind         
        x - state
        u - control input
        v - disturbance
        p - costate
    output:
        H_u, H_ux, H_uu, H_v, H_vx, H_vv, H_uv, H_xx, H_xu, H_xv - derivative of Hammilton with respect to coresponding variables
    '''
    
    H_u = -p[2]*a*np.sin(u) + p[3]*a*np.cos(u)         
    H_uu = -p[2]*a*np.cos(u) - p[3]*a*np.sin(u)    
    H_uv = np.array([0]) 
    H_ux = np.array([.0, .0, .0, .0])
    
    H_v = -p[2]*d*np.sin(v) + p[3]*d*np.cos(v)      
    H_vv = -p[2]*d*np.cos(v) - p[3]*d*np.sin(v) 
    H_vx =  np.array([.0, .0, .0, .0])
    

    H_x = np.array([.0, .0, p[0], p[1]])
    H_xx = np.zeros([4, 4])

    return H_u, H_uu, H_uv, H_ux, H_v, H_vv, H_vx, H_x, H_xx
    

    

In [15]:
"""
    What we do here is really simple. We run as many 
    a fwd and backward pass of ddp algos as possible  
    on trajectories that are finely spaced apart by g.dx 
    
    This part accumulates all trajectories within grid bounds
"""
V_buf = []
all_traj = [copy.deepcopy(grid_min)]
while np.any(all_traj[-1]<grid_max):
    all_traj += [all_traj[-1]+g.dx]

all_traj = np.array(all_traj[:resolution]).squeeze()

# fix the inconsistencies in (x1, x2, x3)
indices = [np.nan for idx in range(len(all_traj))]
for dim in range(all_traj.shape[-1]):
    indices[dim] = all_traj[:,dim]>grid_max[dim,0]
    
# replace trajectories with bounds that exceed max 
# values along respective dimensions
for dim in range(all_traj.shape[-1]):
    all_traj[indices[dim],dim] = grid_max[dim,0]

In [ ]:
x = [1.5, 2.5, 3.0] 
dx = .1

grid_min = [1, 2, 2.5]
grid_max = [2, 3.5, 4.5]

X = [grid_min]
for i < N:
    X += X[-1]+dx

In [7]:
"""
    Once we have the trajectories, we can start with the 
    usual backward and forward pass of DDP.
    
    This will require two loops:
        Upper loop: Whereupon we iterate throuh every possible trajectory in 
        the system
        Lower loop: Forward and backward passes of DDP
"""
x = copy.copy(all_traj)
xr = 
# Here we go:
for traj_idx, traj in enumerate(all_traj):
    

SyntaxError: invalid syntax (<ipython-input-7-dff177466b76>, line 11)

In [39]:
grid_max.T, grid_max.shape

(array([[3.25      , 1.25      , 3.07812614]]), (3, 1))

### Polytope test

Given a matrix $A \in \mathbb{R}^n$ and a vector $b \in \mathbb{R}^m$, an $\mathcal{H}-$ polyhedron is

\begin{align}
    \mathcal{P} = \left\{x \in \mathbb{R}^n \mid Ax \le b \right\}
\end{align}

In [33]:
# x = np.linspace(0.1, 10, 3)

A = np.array([
     [.3, .7, 5.5],
     [8.4, 2.3, 6.7],
     [1.9, 2.5, 8.8]])

b = [1, 2, 3]


# Compute various x's
import numpy.linalg as LA 
Xes = []




array([[3.0000e-02, 3.5350e+00, 5.5000e+01],
       [8.4000e-01, 1.1615e+01, 6.7000e+01],
       [1.9000e-01, 1.2625e+01, 8.8000e+01]])

In [11]:
np.zeros([2,3])

array([[0., 0., 0.],
       [0., 0., 0.]])